In [1]:
#Again we use the "!" command to specify we want to execute commands via the terminal, outside python
#rootls is an example of a root command line tool: 
#https://root.cern/manual/storing_root_objects/#root-command-line-tools
#Allows to run common operations from the linux/mac terminal, without running ROOT or python directly.
! rootls ../Lecture1/tree2.root

t2


In [2]:
#To do the same in python we would have to type:
from ROOT import TFile
myFile = TFile.Open("../Lecture1/tree2.root")
#Note the output is a bit more detailed though
myFile.ls()

Welcome to JupyROOT 6.22/02


In [3]:
#"make" is a standard command to build an executable, which you can run. An executable can be thought of as a 
#translation of your computer code into something the computer can understand. When using python this happens in
#real time, behind the scenes.
#The "!" tells Jupyter to run the command in the underlying terminal.
! make 
#You can see the commands "make" actually runs printed below.
#Lots of options passed to g++ (don't worry about these) - the important part is it creates library files (*.d,*.o)
#and eventually an executable which you run ("myROOTCommand")

g++  -std=c++11 -g -Wall -I -stdlib=libc++ -pthread -std=c++11 -m64 -I/Users/markhodgkinson/build_root_v6_22_02/include -o myROOTCommand.d -MM -MT 'myROOTCommand.o myROOTCommand.d' myROOTCommand.c
clang: warning: treating 'c' input as 'c++' when in C++ mode, this behavior is deprecated [-Wdeprecated]
g++  -std=c++11 -g -Wall -I -stdlib=libc++ -pthread -std=c++11 -m64 -I/Users/markhodgkinson/build_root_v6_22_02/include -o myROOTCommand.o -c myROOTCommand.c
clang: warning: treating 'c' input as 'c++' when in C++ mode, this behavior is deprecated [-Wdeprecated]
g++ -Wl,-rpath,/Users/markhodgkinson/build_root_v6_22_02/lib  -g -Wall -o myROOTCommand myROOTCommand.o  `root-config --libs` -lvectorDict -lTMVA


In [ ]:
#Run the command in the terminal, and we see what it does.
! ./myROOTCommand ../Lecture1/tree2.root t2;1

***********************************************************************************************************
*    Row   * Instance * vect.vect * getot.get * gekin.gek * nmec.nmec * lmec.lmec * destep.de *   pid.pid *
***********************************************************************************************************
*        0 *        0 *         0 * 0.1396002 * 0.0026002 *         0 *           *         0 *         1 *
*        0 *        1 *         0 * 0.1396002 * 0.0026002 *         0 *           *         0 *         1 *
*        0 *        2 *         0 * 0.1396002 * 0.0026002 *         0 *           *         0 *         1 *
*        0 *        3 * 0.7449554 * 0.1396002 * 0.0026002 *         0 *           *         0 *         1 *
*        0 *        4 * -0.324226 * 0.1396002 * 0.0026002 *         0 *           *         0 *         1 *
*        0 *        5 * 0.5830255 * 0.1396002 * 0.0026002 *         0 *           *         0 *         1 *
*        0 *        6 * -0.0

In [ ]:
#Task 1
#Modify myROOTCommand.C (use any text editor on your computer)